In [77]:
import pandas as pd
from sklearn.model_selection import train_test_split, RandomizedSearchCV, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from scipy.stats import uniform, randint

In [95]:
# Load the data
file_path = 'train data.csv'
data = pd.read_csv(file_path)
batsman_data = pd.read_csv('batsman.csv')
batsman_data = batsman_data.ffill()
bowler_data = pd.read_csv('bowler.csv')
bowler_data = bowler_data.ffill()
test_data = pd.read_csv('test data.csv')
test_data = test_data.ffill()

In [96]:
'''
def calculate_average_strike_rate(roster, match_id, batsman_data):
    batsman_ids = [int(float(bid)) for bid in roster.split(':')]
    strike_rates = batsman_data[(batsman_data['match id'] == match_id) & (batsman_data['batsman_id'].isin(batsman_ids))]['strike_rate']
    return strike_rates.mean() if not strike_rates.empty else 0
def calculate_average_economy(roster, match_id, bowler_data):
    bowler_ids = [int(float(bid)) for bid in roster.split(':')]
    economy_rates = bowler_data[(bowler_data['match id'] == match_id) & (bowler_data['bowler_id'].isin(bowler_ids))]['economy']
    return economy_rates.mean() if not economy_rates.empty else 0
# Add new columns for average strike rates
data['team1_avg_strike_rate'] = data.apply(lambda row: calculate_average_strike_rate(row['team1_roster_ids'], row['match id'], batsman_data), axis=1)
data['team2_avg_strike_rate'] = data.apply(lambda row: calculate_average_strike_rate(row['team2_roster_ids'], row['match id'], batsman_data), axis=1)
data['team1_avg_economy'] = data.apply(lambda row: calculate_average_economy(row['team1_roster_ids'], row['match id'], bowler_data), axis=1)
data['team2_avg_economy'] = data.apply(lambda row: calculate_average_economy(row['team2_roster_ids'], row['match id'], bowler_data), axis=1)
test_data['team1_avg_strike_rate'] = test_data.apply(lambda row: calculate_average_strike_rate(row['team1_roster_ids'], row['match id'], batsman_data), axis=1)
test_data['team2_avg_strike_rate'] = test_data.apply(lambda row: calculate_average_strike_rate(row['team2_roster_ids'], row['match id'], batsman_data), axis=1)
test_data['team1_avg_economy'] = test_data.apply(lambda row: calculate_average_economy(row['team1_roster_ids'], row['match id'], bowler_data), axis=1)
test_data['team2_avg_economy'] = test_data.apply(lambda row: calculate_average_economy(row['team2_roster_ids'], row['match id'], bowler_data), axis=1)
'''

"\ndef calculate_average_strike_rate(roster, match_id, batsman_data):\n    batsman_ids = [int(float(bid)) for bid in roster.split(':')]\n    strike_rates = batsman_data[(batsman_data['match id'] == match_id) & (batsman_data['batsman_id'].isin(batsman_ids))]['strike_rate']\n    return strike_rates.mean() if not strike_rates.empty else 0\ndef calculate_average_economy(roster, match_id, bowler_data):\n    bowler_ids = [int(float(bid)) for bid in roster.split(':')]\n    economy_rates = bowler_data[(bowler_data['match id'] == match_id) & (bowler_data['bowler_id'].isin(bowler_ids))]['economy']\n    return economy_rates.mean() if not economy_rates.empty else 0\n# Add new columns for average strike rates\ndata['team1_avg_strike_rate'] = data.apply(lambda row: calculate_average_strike_rate(row['team1_roster_ids'], row['match id'], batsman_data), axis=1)\ndata['team2_avg_strike_rate'] = data.apply(lambda row: calculate_average_strike_rate(row['team2_roster_ids'], row['match id'], batsman_data), 

In [97]:
def calculate_average_strike_rate(roster, batsman_data):
    batsman_ids = [int(float(bid)) for bid in roster.split(':')]
    strike_rates = batsman_data[batsman_data['batsman_id'].isin(batsman_ids)]['strike_rate']
    return strike_rates.mean() if not strike_rates.empty else 0
def calculate_average_economy(roster, bowler_data):
    bowler_ids = [int(float(bid)) for bid in roster.split(':')]
    economy_rates = bowler_data[bowler_data['bowler_id'].isin(bowler_ids)]['economy']
    return economy_rates.mean() if not economy_rates.empty else 0
# Add new columns for average strike rates
data['team1_avg_strike_rate'] = data.apply(lambda row: calculate_average_strike_rate(row['team1_roster_ids'], batsman_data), axis=1)
data['team2_avg_strike_rate'] = data.apply(lambda row: calculate_average_strike_rate(row['team2_roster_ids'], batsman_data), axis=1)
data['team1_avg_economy'] = data.apply(lambda row: calculate_average_economy(row['team1_roster_ids'], bowler_data), axis=1)
data['team2_avg_economy'] = data.apply(lambda row: calculate_average_economy(row['team2_roster_ids'], bowler_data), axis=1)
test_data['team1_avg_strike_rate'] = test_data.apply(lambda row: calculate_average_strike_rate(row['team1_roster_ids'], batsman_data), axis=1)
test_data['team2_avg_strike_rate'] = test_data.apply(lambda row: calculate_average_strike_rate(row['team2_roster_ids'], batsman_data), axis=1)
test_data['team1_avg_economy'] = test_data.apply(lambda row: calculate_average_economy(row['team1_roster_ids'],  bowler_data), axis=1)
test_data['team2_avg_economy'] = test_data.apply(lambda row: calculate_average_economy(row['team2_roster_ids'], bowler_data), axis=1)

In [99]:
test_data.head()

,match id,team1,team1_id,team1_roster_ids,team2,team2_id,team2_roster_ids,toss winner,toss decision,venue,...,ground_id,team_count_50runs_last15,team_winp_last5,team1only_avg_runs_last15,team1_winp_team2_last15,ground_avg_runs_last15,team1_avg_strike_rate,team2_avg_strike_rate,team1_avg_economy,team2_avg_economy
0,9250275,Jy,28594,7438955.0:8271969.0:8369661.0:3685247.0:259025...,Ud Ss of Aa,90,2436944.0:8117500.0:6703528.0:3010748.0:161387...,Ud Ss of Aa,field,Bo Ac Cb,...,3226,0.000000,0.019608,NaN,0.00,NaN,127.696250,110.605610,6.707500,8.436944
1,9262189,Ga An Ws,36084,3715690.0:6818622.0:4069666.0:393014.0:4690188...,Ja Ts,36098,4690258.0:3761246.0:8464385.0:5742470.0:315072...,Ja Ts,field,Pe Sm Ga,...,13915,0.615385,0.344262,151.285714,66.67,153.500,120.895835,115.088116,8.094824,7.917769
2,9128776,Rn Rs,30428,8058959.0:2162782.0:2981614.0:7833195.0:755605...,Lw Sr Gs,48334,2654014.0:2954769.0:8058903.0:3479860.0:329940...,Rn Rs,bat,Be Sm Mi,...,2764,0.842105,0.753086,171.066667,100.00,179.625,119.908054,115.264369,8.485485,8.224095
3,9586919,Ja Ts,36098,4690258.0:8464385.0:1613898.0:5744780.0:315072...,St La Ks,36112,2331475.0:313809.0:391103.0:2742837.0:2097017....,St La Ks,field,Dn Sy Nl Ct Sm Gs It St La,...,11892,0.285714,1.487805,166.000000,75.00,155.500,110.809852,116.571064,7.582842,8.071719
4,9128538,Ci Sr Ks,30414,7422673.0:249087.0:62432.0:232000.0:5001170.0:...,Gt Ts,48341,1958683.0:7491224.0:3339160.0:5452754.0:225245...,Gt Ts,field,Ma Ct An Sm Pe,...,18752,2.375000,0.310330,169.933333,0.00,164.125,121.387657,112.850781,8.109003,7.757143


In [100]:
columns_to_drop_data = ['match id', 'team1_roster_ids', 'team2_roster_ids', 'match_dt', 'winner_id']
columns_to_drop_test_data = ['match id', 'team1_roster_ids', 'team2_roster_ids', 'match_dt']
print(columns_to_drop_data)
print(columns_to_drop_test_data)
print(data.columns)
print(test_data.columns)
data = data.drop(columns=columns_to_drop_data)

# Drop the columns from test_data
test_data = test_data.drop(columns=[col for col in columns_to_drop_test_data if col in test_data.columns])


['match id', 'team1_roster_ids', 'team2_roster_ids', 'match_dt', 'winner_id']
['match id', 'team1_roster_ids', 'team2_roster_ids', 'match_dt']
Index(['match id', 'team1', 'team1_id', 'team1_roster_ids', 'team2',
       'team2_id', 'team2_roster_ids', 'winner', 'winner_id', 'toss winner',
       'toss decision', 'venue', 'city', 'match_dt', 'lighting', 'series_name',
       'season', 'ground_id', 'team_count_50runs_last15', 'team_winp_last5',
       'team1only_avg_runs_last15', 'team1_winp_team2_last15',
       'ground_avg_runs_last15', 'team1_avg_strike_rate',
       'team2_avg_strike_rate', 'team1_avg_economy', 'team2_avg_economy'],
      dtype='object')
Index(['match id', 'team1', 'team1_id', 'team1_roster_ids', 'team2',
       'team2_id', 'team2_roster_ids', 'toss winner', 'toss decision', 'venue',
       'city', 'match_dt', 'lighting', 'series_name', 'season', 'ground_id',
       'team_count_50runs_last15', 'team_winp_last5',
       'team1only_avg_runs_last15', 'team1_winp_team2_la

In [101]:
# Handle missing values
imputer = SimpleImputer(strategy='median')
data[['team1only_avg_runs_last15', 'ground_avg_runs_last15']] = imputer.fit_transform(data[['team1only_avg_runs_last15', 'ground_avg_runs_last15']])
test_data[['team1only_avg_runs_last15', 'ground_avg_runs_last15']] = imputer.fit_transform(test_data[['team1only_avg_runs_last15', 'ground_avg_runs_last15']])

In [102]:
# Encode categorical variables
label_encoder = LabelEncoder()

In [104]:
# Binary categorical variables
data['winner'] = data.apply(lambda row: 0 if row['winner'] == row['team1'] else 1, axis=1)
data['toss winner'] = data.apply(lambda row: 0 if row['toss winner'] == row['team1'] else 1, axis=1)
data['toss decision'] = label_encoder.fit_transform(data['toss decision'])
data['lighting'] = label_encoder.fit_transform(data['lighting'])

test_data['toss winner'] = test_data.apply(lambda row: 0 if row['toss winner'] == row['team1'] else 1, axis=1)
test_data['toss decision'] = label_encoder.fit_transform(test_data['toss decision'])
test_data['lighting'] = label_encoder.fit_transform(test_data['lighting'])

In [105]:
categorical_columns = data.select_dtypes(include=['object']).columns
categorical_columns

Index(['team1', 'team2', 'venue', 'city', 'series_name', 'season'], dtype='object')

In [106]:
label_encoders = {}
for col in categorical_columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

In [107]:
test_label_encoders = {}
for col in categorical_columns:
    test_le = LabelEncoder()
    test_data[col] = test_le.fit_transform(test_data[col])
    test_label_encoders[col] = test_le

In [108]:
print(data)

     team1  team1_id  team2  team2_id  winner  toss winner  toss decision  \
0        6     11283     43     12634       1            1              1   
1       35        20    138        41       1            1              1   
2      135     10576     81      8987       1            1              1   
3       39     36084     13     36070       1            1              0   
4       96     30407     38     48341       1            1              1   
..     ...       ...    ...       ...     ...          ...            ...   
943     96     30407     15     30414       1            1              1   
944     81      8700    114      9701       1            1              1   
945      7     22497    136     23869       1            1              0   
946    105     30428    113     36014       1            1              0   
947     47      7727     33      7608       1            1              1   

     venue  city  lighting  ...  ground_id  team_count_50runs_last15  \
0  

In [109]:
# Multi-class categorical variables
#multi_class_cols = ['team1', 'team2', 'venue', 'city', 'series_name', 'season']
#data = pd.get_dummies(data, columns=multi_class_cols)

In [110]:
X_train = data.drop(['winner'], axis=1)
y_train = data['winner']
X_test = test_data
#y_train = test_data


In [111]:
print(y_train)

0      1
1      1
2      1
3      1
4      1
      ..
943    1
944    1
945    1
946    1
947    1
Name: winner, Length: 948, dtype: int64


In [112]:
print(X_train.shape)  # Should be (948, n_features)
print(y_train.shape)  # Should be (948,)


(948, 21)
(948,)


In [113]:
# Define the parameter grid
param_grid = {
    'max_depth': [3, 5, 7],
    'learning_rate': [0.01, 0.1, 0.5],
    'n_estimators': [50, 100, 200],
    'gamma': [0, 0.1, 0.5],
    'subsample': [0.5, 0.8, 1],
    'colsample_bytree': [0.5, 0.8, 1],
    'reg_alpha': [0, 0.1, 0.5],
    'reg_lambda': [0, 0.1, 0.5]
}

In [114]:
# Initialize the model
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss')

In [115]:
# Perform grid Search
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

ValueError: 
All the 32805 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
32805 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Ritika\anaconda3\envs\cricket\Lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Ritika\anaconda3\envs\cricket\Lib\site-packages\xgboost\core.py", line 730, in inner_f
    return func(**kwargs)
           ^^^^^^^^^^^^^^
  File "C:\Users\Ritika\anaconda3\envs\cricket\Lib\site-packages\xgboost\sklearn.py", line 1471, in fit
    raise ValueError(
ValueError: Invalid classes inferred from unique values of `y`.  Expected: [0], got [1]


In [ ]:
# Best model
best_model = grid_search.best_estimator_

In [ ]:
# Make predictions
y_pred_train = best_model.predict(X_train)
y_pred_test = best_model.predict(X_test)

In [ ]:
# Evaluate the model
accuracy = accuracy_score(y_train, y_pred_train)
print(f'Accuracy: {accuracy * 100:.2f}%')

In [ ]:

# Print feature importances
feature_importances = best_model.feature_importances_
feature_names = X.columns



In [ ]:
# Create a DataFrame with feature names and their corresponding importances
importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': feature_importances})


In [ ]:

# Sort the DataFrame by importance in descending order
importance_df = importance_df.sort_values('Importance', ascending=False)

# Print the feature importances
print("Feature Importances:")
print(importance_df)